<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_04_Summarize_Podcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to ingest a podcast MP3 by URL, and summarize in various formats, including generating chapters.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [ ]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [ ]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [ ]:
from typing import List, Optional

async def ingest_uri(uri: str):
    if graphlit.client is None:
        return;

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def summarize_audio(prompt: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.summarize_contents(
            filter=input_types.ContentFilter(
                # Filter just on audio content
                fileTypes=[enums.FileTypes.AUDIO],
            ),
            summarizations=[
                input_types.SummarizationStrategyInput(
                    type=enums.SummarizationTypes.CHAPTERS,
                ),
                input_types.SummarizationStrategyInput(
                    type=enums.SummarizationTypes.CUSTOM,
                    prompt=prompt
                ),
            ]
        )

        return response.summarize_contents
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)


Execute Graphlit example

In [ ]:
from IPython.display import display, Markdown

# Remove any existing contents; only needed for notebook example
await delete_all_contents()

print('Deleted all contents.')

content_id = await ingest_uri(uri="https://graphlitplatform.blob.core.windows.net/samples/Podcasts/GraphRAG%20Knowledge%20Graphs%20for%20AI%20Applications%20with%20Kirk%20Marple.mp3")

print(f'Ingested content [{content_id}].')

prompt = "What is one key takeaway from this podcast? Explain thoroughly."

summarizations = await summarize_audio(prompt)

if summarizations is not None:
    for summarization in summarizations:
        if summarization is not None and summarization.items is not None:
            print(f'Summarization [{summarization.type}]:')

            for item in summarization.items:
                display(Markdown(item.text))

            print()